Importamos las librerias necesarias

In [1]:
import pandas as pd
import os

Obtenemos la ruta de la carpeta en la que estamos trabajando

Nos servira para abrir los archivos y luego guardarlos

In [2]:
# Construir la ruta completa al archivo
ruta_review = r'C:\PROYECTO-INDIVIDUAL\Data\json\sucio\australian_user_reviews.json'

# Abrimos el archivo y cargamos todas las lineas del mismo en "lineas"
with open(ruta_review, 'r', encoding='utf-8') as archivo:
    lineas = archivo.readlines()

#leemos cada linea dentro de lineas, y realizamos la correcciones pertinentes
texto = []
for linea in lineas:
    texto_de_linea = linea.replace("'", '"').replace('"', '').replace('user_id: ',  '"user_id": "').replace(', user_url: ',  '", "user_url": "').replace(', reviews:',  '", "reviews":').replace('funny:',  '"funny": "').replace(', posted:',  '", "posted": "').replace(', last_edited:',  '", "last_edited": "').replace(', item_id:',  '", "item_id": "').replace(', helpful:',  '", "helpful": "').replace(', recommend:',  '", "recommend": "').replace(', review:',  '", "review": "').replace('},', '"},').replace('}]', '"}]').replace("\\", "")
    texto.append(texto_de_linea)

# Abrimos el archivo en formato escritura, cargamos todo el texto, y guardamos en una nueva ruta
ruta_nueva_reviews = r'C:\PROYECTO-INDIVIDUAL\Data\json\limpio\reviews.json'
archivo = open(ruta_nueva_reviews, "w", encoding='utf-8')
archivo.write('[' + '\n')
for i in range(len(texto)-1):
    archivo.write(texto[i] + ',')
archivo.write(texto[len(texto)-1])
archivo.write(']' + '\n')
archivo.close()

CREAMOS UN DATAFRAME CON ESE ARCHIVO

In [3]:
df_reviews = pd.read_json(ruta_nueva_reviews, encoding='utf-8')

REALIZAMOS UNA VISUALIZACION DE LOS DATOS, FILAS Y COLUMNAS

Con el fin de saber que columnas nos pueden servir de acuerdo a las actividades a realizar

In [4]:
print(df_reviews.shape)
print(df_reviews.columns)

(25799, 3)
Index(['user_id', 'user_url', 'reviews'], dtype='object')


AHORA ELIMINAREMOS COLUMNAS, VALORES NA, Y MODIFICAREMOS DATOS

In [5]:
# Separar la columna reviews
df_reviews = df_reviews.explode(column='reviews')

# Creamos listas donde se guardara todos los datos de lo que sera el nuevo df
lista_item_id = []
lista_helpful = []
lista_recommend = []
lista_review = []
for x in range(len(df_reviews['reviews'])):
    review = df_reviews['reviews'].iloc[x]
    if pd.notna(review):
        diccionario = dict(review)
        if 'helpful' in diccionario:
            lista_helpful.append(diccionario['helpful'])
        else:
            lista_helpful.append('None')
        if 'item_id' in diccionario:
            lista_item_id.append(diccionario['item_id'])
        else:
            lista_item_id.append('None')
        if 'recommend' in diccionario:
            lista_recommend.append(diccionario['recommend'])
        else:
            lista_recommend.append('None')
        if 'review' in diccionario:
            lista_review.append(diccionario['review'])
        else:
            lista_review.append('None')
    else:
        lista_helpful.append('None')
        lista_item_id.append('None')
        lista_recommend.append('None')
        lista_review.append('None')


# Creamos las nuevas columnas con las listas hechas anteriormente
df_reviews['item_id'] = lista_item_id
df_reviews['helpful'] = lista_helpful
df_reviews['recommend'] = lista_recommend
df_reviews['review'] = lista_review

# Dropeamos las columnas que no necesitamos
df_reviews.drop(columns='reviews', inplace=True)
df_reviews.drop(columns='user_url', inplace=True)
df_reviews.drop(columns='helpful', inplace=True)

# Creamos una lista con las recomendaciones del df, 2 = Bueno, 1 = Neutral 0 = Malo
columna_recomendacion = []
for recomendacion, review in zip(df_reviews['recommend'], df_reviews['review']):
    if recomendacion.strip() == 'True':
        columna_recomendacion.append(2)
    elif recomendacion.strip() == 'False':
        columna_recomendacion.append(0)
    elif review.strip() == '' or review.strip() == 'None':
        columna_recomendacion.append(1)
    elif recomendacion.strip() == 'None' or recomendacion.strip() == '':
        columna_recomendacion.append(1)
    else:
        columna_recomendacion.append(1)

# Reemplzamos la columna de recomendacion por la lista que creamos
df_reviews['recommend'] = columna_recomendacion
df_reviews['recommend'] = columna_recomendacion

df_reviews['user_id'] = df_reviews['user_id'].str.strip()
df_reviews['item_id'] = df_reviews['item_id'].str.strip()
df_reviews.loc[df_reviews['item_id'].astype(str).str.contains('None'), 'item_id'] = None
df_reviews.dropna(subset=['item_id'], inplace=True)
df_reviews['item_id'] = df_reviews['item_id'].astype(int)

Vemos como quedo

In [6]:
df_reviews.head()

,user_id,item_id,recommend,review
0,76561197970982479,1250,2,Simple yet with great replayability. In my op...
0,76561197970982479,22200,2,Its unique and worth a playthrough.
0,76561197970982479,43110,2,Great atmosphere. The gunplay can be a bit ch...
1,js41637,251610,2,I know what you think when you see this title...
1,js41637,227300,2,For a simple (its actually not all that simpl...


GUARDAMOS LOS DATAFRAMES

Para ello creamos una carpeta llamada parquet dentro de datos, y guardamos los archivos dentro de ella

In [7]:
ruta_de_guardado = r'C:\PROYECTO-INDIVIDUAL\Data\parquet\reviews.parquet'
df_reviews.to_parquet(ruta_de_guardado, index=False)